In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
! pip install streamlit -q

     |████████████████████████████████| 9.1 MB 4.9 MB/s 
     |████████████████████████████████| 235 kB 56.5 MB/s 
     |████████████████████████████████| 181 kB 49.6 MB/s 
     |████████████████████████████████| 4.3 MB 30.3 MB/s 
     |████████████████████████████████| 164 kB 46.3 MB/s 
     |████████████████████████████████| 78 kB 6.4 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 132 kB 56.4 MB/s 
     |████████████████████████████████| 793 kB 49.9 MB/s 
     |████████████████████████████████| 132 kB 81.1 MB/s 
     |████████████████████████████████| 423 kB 53.0 MB/s 
     |████████████████████████████████| 381 kB 52.6 MB/s 
     |████████████████████████████████| 51 kB 6.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.6.6 requires traitlets>=5.2.2, but you have traitlets 5.1.1 which i

In [3]:
%%writefile app.py
import streamlit as st

st.title("Food Recommendation System")
st.text("Let us help you with ordering")
st.image("foodimage.jpg")

## nav = st.sidebar.radio("Navigation",["Home","IF Necessary 1","If Necessary 2"])

st.subheader("Whats your preference?")
vegn = st.radio("Vegetables or none!",["veg","non-veg"],index = 1) 

st.subheader("What Cuisine do you prefer?")
cuisine = st.selectbox("Choose your favourite!",['Healthy Food', 'Snack', 'Dessert', 'Japanese', 'Indian', 'French',
       'Mexican', 'Italian', 'Chinese', 'Beverage', 'Thai'])

st.subheader("How well do you want the dish to be?")  #RATING
val = st.slider("from poor to the best!",0,10)

import pandas as pd
food = pd.read_csv(r"food.csv")
ratings = pd.read_csv(r"ratings.csv")
combined = pd.merge(ratings, food, on='Food_ID')
#ans = food.loc[(food.C_Type == cuisine) & (food.Veg_Non == vegn),['Name','C_Type','Veg_Non']]
ans = combined.loc[(combined.C_Type == cuisine) & (combined.Veg_Non == vegn)& (combined.Rating >= val),['Name','C_Type','Veg_Non']]
names = ans['Name'].tolist()

import numpy as np
x = np.array(names)
ans1 = np.unique(x)

finallist = ""
dish = st.checkbox("Choose your Dish")
if dish == True:
    finallist = st.selectbox("Our Choices",ans1)


##### IMPLEMENTING RECOMMENDER ######
dataset = ratings.pivot_table(index='Food_ID',columns='User_ID',values='Rating')
dataset.fillna(0,inplace=True)

from scipy.sparse import csr_matrix
csr_dataset = csr_matrix(dataset.values)
dataset.reset_index(inplace=True)

from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
model.fit(csr_dataset)

def food_recommendation(Food_Name):
    n = 10
    FoodList = food[food['Name'].str.contains(Food_Name)]  
    if len(FoodList):        
        Foodi= FoodList.iloc[0]['Food_ID']
        Foodi = dataset[dataset['Food_ID'] == Foodi].index[0]
        distances , indices = model.kneighbors(csr_dataset[Foodi],n_neighbors=n+1)    
        Food_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        Recommendations = []
        for val in Food_indices:
            Foodi = dataset.iloc[val[0]]['Food_ID']
            i = food[food['Food_ID'] == Foodi].index
            Recommendations.append({'Name':food.iloc[i]['Name'].values[0],'Distance':val[1]})
        df = pd.DataFrame(Recommendations,index=range(1,n+1))
        return df['Name']
    else:
        return "No Similar Foods."


display = food_recommendation(finallist)
#names1 = display['Name'].tolist()

#x1 = np.array(names)
#ans2 = np.unique(x1)
if dish == True:
    dish1 = st.checkbox("Other dishes with the same rating as you prefered : ")
    if dish1 == True:
        for i in display:
            st.write(i)

Writing app.py


In [4]:
 ! pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 5.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=1455abd02548b3b01e7d3ee016bf0447e4a78effaceb8f302a80e064c7312bbc
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [5]:
from pyngrok import ngrok

In [6]:
ngrok.set_auth_token("2CCLE9yJmYV0kO1BLs9TsVCqyKr_3fgd6FL1Hd64oVchQYvqX")

In [7]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://0c0d-35-231-249-148.ngrok.io" -> "http://localhost:80"


In [8]:
from pyngrok import ngrok

tunnels = ngrok.get_tunnels()
tunnels

# try:
#     # Block until CTRL-C or some other terminating event
#     ngrok_process.proc.wait()
# except KeyboardInterrupt:
#     print(" Shutting down server.")

#     ngrok.kill()

[<NgrokTunnel: "http://0c0d-35-231-249-148.ngrok.io" -> "http://localhost:80">,
 <NgrokTunnel: "https://0c0d-35-231-249-148.ngrok.io" -> "http://localhost:80">]